<a href="https://colab.research.google.com/github/Cours-EDUlib/FAS-ISDS/blob/main/module-7/Exercices/7_2_solution_exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Module 7: Les statistiques et la science des données appliquées
##Exercice 2
###Solutionnaire

Reprenez l'exemple de régression simple testant l'hypothèse que la taille du réseau professionnel est associée au statut socioéconomique, en tenant compte des données extrêmes.

In [ ]:
#Téléchargement des packages nécessaires
library (tidyverse)
install.packages("rsq")
require(rsq)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘statmod’, ‘RcppEigen’, ‘lme4’, ‘numDeriv’


Loading required package: rsq



In [ ]:
path_to_csv = 'https://github.com/Cours-EDUlib/FAS-ISDS/blob/main/module-7/Exercices/7-1-2_donnees.csv?raw=true' #Téléchargement de la base de données depuis github

bd <- read_csv(path_to_csv) #Enregistrement de la base de données dans l'ojet "bd"
bd #Lecture des données

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  id = col_double(),
  ses = col_double(),
  genre = col_double(),
  reseau = col_double(),
  scolarite = col_double(),
  experience = col_double()
)




X1,id,ses,genre,reseau,scolarite,experience
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,62.50000,2,24,2,6
2,5,17.10000,2,10,0,1
3,6,69.20000,2,13,3,12
4,7,29.80000,1,4,0,4
5,8,37.70000,2,24,1,4
6,9,37.70000,2,6,1,6
7,10,47.85451,2,14,2,9
8,11,92.30000,2,9,4,27
9,12,80.90000,2,5,4,14


In [ ]:
#Statistiques descriptives, histogramme et nuage de points faits dans exercice 7.1

#Régression linéaire sans retrait des données extrêmes
reglineaire <- glm(experience ~ ses, data = bd, family="gaussian")
summary(rstandard(reglineaire))
summary(reglineaire)
rsq(reglineaire)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  17.10   32.40   40.20   47.85   63.50   97.20 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0.00    6.00   10.00   10.37   14.00   32.00 

[1] 18.58611

[1] 6.041134

In [ ]:
#Identifier données extrêmes
outliers <- bd$id[rstandard(reglineaire) < -3 | rstandard(reglineaire) > 3]
outliers

[1]   36 2303 2358 2451 2729 2807  135 2593 2831  310  390  417  610  773  785
[16]  801  921 1009 1043 1044 1126 1168 1539 1729 1853 1929 1930 2051 2107 1003
[31] 1405 1463 1762  998 1382 1711 1750 1955 1977  475

In [ ]:
#Retirer données extrêmes
bd2 <- bd [-outliers,]
bd2

X1,id,ses,genre,reseau,scolarite,experience
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,62.50000,2,24,2,6
2,5,17.10000,2,10,0,1
3,6,69.20000,2,13,3,12
4,7,29.80000,1,4,0,4
5,8,37.70000,2,24,1,4
6,9,37.70000,2,6,1,6
7,10,47.85451,2,14,2,9
8,11,92.30000,2,9,4,27
9,12,80.90000,2,5,4,14


In [ ]:
#Refaire régression linéaire après le retrait des données extrêmes

reglineaire2 <- glm(experience ~ ses, data = bd2, family="gaussian")
summary(rstandard(reglineaire2))
summary(reglineaire2)
rsq(reglineaire2)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-3.230482 -0.533171 -0.089580  0.000027  0.362653  5.604756 


Call:
glm(formula = experience ~ ses, family = "gaussian", data = bd2)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-13.2472   -2.1871   -0.3674    1.4863   22.9838  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -6.133249   0.211568  -28.99   <2e-16 ***
ses          0.329911   0.004116   80.16   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 16.83293)

    Null deviance: 156325  on 2863  degrees of freedom
Residual deviance:  48176  on 2862  degrees of freedom
AIC: 16218

Number of Fisher Scoring iterations: 2


[1] 0.691822